In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from FaceFeatureDataset import FaceFeatureDataset
import dlib_index as DI
import pandas as pd
import numpy as np

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
batch_size = 10
epochs = 500

In [5]:
features = pd.read_csv('./outimg/Train/facefeature.csv')
features = features.iloc[0, 1:]
features = np.array(features, dtype=np.float32).reshape(-1,2);
features = np.take(features, DI.LEFT_EYE, axis=0)
print(features)


[[ 0.10154363 -0.1885906 ]
 [ 0.14489932 -0.22053692]
 [ 0.19510067 -0.22281879]
 [ 0.2361745  -0.20456375]
 [ 0.19966443 -0.18174496]
 [ 0.14946309 -0.17946309]]


In [4]:
# 머리      headSize =0, headWidth = 1,
# 이마      foreheadPosition= 2,foreheadSize = 3,
# 턱        jawsPosition =4,jawsSize = 5,
# 아래턱    chinPosition = 6,chinSize = 7,chinPronounced = 8,
# 뺨        lowCheek = 9, cheekPosition = 10, cheekSize = 11,
# 귀        earsPosition = 12, earsRotation =13, earsSize = 14
# 눈        eyeSize = 15, eyePosition= 16, eyeDepth = 17, eyeRotation = 18, eyeDistance = 19, eyeSquint= 20,
# 코        noseSize,nosePosition,noseFlatten,nosePronounced,noseWidth,noseBridge,noseCurve,noseInclination,
# 입        mouthSize,mouthPosition,mouthPronounced,lipsSize

eyeIndexes = ['eyeSize', 'eyePosition', 'eyeDepth', 'eyeRotation', 'eyeDistance', 'eyeSquint']
#eyeIndexes = list(range(15, 21))

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", feature_indexes=DI.LEFT_EYE,
                                   label_file="./Dataset/Train/csv/train.csv", label_indexes=eyeIndexes)
test_data = FaceFeatureDataset(
    feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인
for X, y in train_loader:
    # N , Channel, H= width W = height
    print(f"Shape of X [N, F, C]: {X.shape}")
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")
    break

n_total_steps = len(train_loader)


[[ 0.10154363 -0.1885906 ]
 [ 0.14489932 -0.22053692]
 [ 0.19510067 -0.22281879]
 [ 0.2361745  -0.20456375]
 [ 0.19966443 -0.18174496]
 [ 0.14946309 -0.17946309]]
[[ 0.10061225 -0.19306122]
 [ 0.14455782 -0.2185034 ]
 [ 0.19312926 -0.2185034 ]
 [ 0.23707484 -0.20462584]
 [ 0.20006803 -0.1814966 ]
 [ 0.14918368 -0.1814966 ]]
[[ 0.09998894 -0.1884398 ]
 [ 0.1412192  -0.22190624]
 [ 0.19430037 -0.22635248]
 [ 0.23756564 -0.20675485]
 [ 0.20120913 -0.18048787]
 [ 0.14812796 -0.17604165]]
[[ 0.10433789 -0.19365232]
 [ 0.14599673 -0.21977147]
 [ 0.19216487 -0.2215405 ]
 [ 0.23073658 -0.2015609 ]
 [ 0.19643134 -0.18019384]
 [ 0.1502979  -0.18085289]]
[[ 0.09296875 -0.189375  ]
 [ 0.14078125 -0.2265625 ]
 [ 0.19921875 -0.22921875]
 [ 0.244375   -0.2       ]
 [ 0.20453125 -0.1734375 ]
 [ 0.14609376 -0.1734375 ]]
[[ 0.10243012 -0.19049683]
 [ 0.14516678 -0.2226826 ]
 [ 0.19516408 -0.22231497]
 [ 0.23497811 -0.19702221]
 [ 0.19983295 -0.17728063]
 [ 0.14733578 -0.17766665]]
[[ 0.09309501 -0.18581

In [5]:
num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)


In [6]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
# Training
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
[[ 0.35026845 -0.12241611]
 [ 0.36167786 -0.21597315]
 [-0.29322147 -0.2867114 ]
 [-0.2475839  -0.31637585]
 [-0.18597315 -0.3209396 ]
 [-0.13120805 -0.30724832]]
[[ 0.34809524 -0.10517007]
 [ 0.35965985 -0.19537415]
 [-0.2995238  -0.27632654]
 [-0.2532653  -0.3110204 ]
 [-0.19312926 -0.3179592 ]
 [-0.13530612 -0.30408162]]
[[ 0.35997385 -0.12631224]
 [ 0.3726969  -0.22026455]
 [-0.30439594 -0.29017302]
 [-0.25353786 -0.32115984]
 [-0.19322301 -0.32555476]
 [-0.13545619 -0.310677  ]]
[[ 0.36275455 -0.09281779]
 [ 0.3738543  -0.18980208]
 [-0.30229545 -0.2893185 ]
 [-0.255607   -0.32750866]
 [-0.19240767 -0.33146298]
 [-0.13673536 -0.31852478]]
[[ 0.40109375 -0.09375   ]
 [ 0.414375   -0.2053125 ]
 [-0.31875    -0.295625  ]
 [-0.26828125 -0.33546874]
 [-0.201875   -0.34078124]
 [-0.14078125 -0.3275    ]]
[[ 0.3641623  -0.08607233]
 [ 0.37484187 -0.17849381]
 [-0.30692273 -0.2785068 ]
 [-0.25913122 -0.31815538]
 [-0.19656107 -0.3276953 ]
 [-0.13415

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x12 and 136x128)

In [ ]:
PATH = './face_eye.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [ ]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4949, 0.4872, 0.4981, 0.5107, 0.5251, 0.5245, 0.4764, 0.5042, 0.5239,
         0.5138, 0.5233, 0.5014, 0.4990, 0.5154, 0.4866, 0.4953, 0.5167, 0.5367,
         0.4967, 0.5231, 0.5004, 0.5102, 0.5002, 0.5012, 0.5219, 0.4913, 0.5280,
         0.4912, 0.4862, 0.5077, 0.5270, 0.5217, 0.4868],
        [0.4914, 0.4847, 0.4956, 0.5111, 0.5210, 0.5206, 0.4752, 0.5010, 0.5215,
         0.5101, 0.5213, 0.4998, 0.4949, 0.5150, 0.4840, 0.4918, 0.5138, 0.5364,
         0.4955, 0.5235, 0.4979, 0.5055, 0.4957, 0.4990, 0.5188, 0.4893, 0.5253,
         0.4886, 0.4824, 0.5050, 0.5258, 0.5178, 0.4822],
        [0.4968, 0.4900, 0.5007, 0.5134, 0.5280, 0.5262, 0.4786, 0.5064, 0.5254,
         0.5142, 0.5253, 0.5038, 0.5012, 0.5185, 0.4882, 0.4979, 0.5183, 0.5407,
         0.4995, 0.5255, 0.5018, 0.5123, 0.5016, 0.5020, 0.5233, 0.4949, 0.5300,
         0.4924, 0.4885, 0.5092, 0.5303, 0.5236, 0.4890],
        [0.4972, 0.4910, 0.5013, 0.5126, 0.5294, 0.5266, 0.4777, 0.5069, 0.5